In [1]:
import os
import math
import random

import geopandas as gpd
import pandas as pd
import numpy as np

from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

/home/danbot/Documents/code/data_analysis/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


Loading BokehJS ...

In [2]:
def make_plot(title, hist, edges):
    p = figure(title=title, tools='wheel_zoom,reset,box_zoom', background_fill_color="#fafafa")
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color="navy", line_color="white", alpha=0.5)
    p.y_range.start = 0
    p.xaxis.axis_label = 'x'
    p.yaxis.axis_label = 'Pr(x)'
    p.grid.grid_line_color="white"
    return p

In [3]:
def draw_a_jack():
    choices = [1, 0]
    for m in range(51):
        pj, pnj, pdraw = calc_p(m)
        # print(f'PJ:{pj:.2f}  PnJ:{pnj:.3f}')
        card = random.choices(choices, weights=[pj, pnj], k=1)[0]
        if card == 1:
            # print(f'drew a card in {m}')
            return m
    

In [73]:
n_draws = [draw_a_jack() for i in range(1000000)]

In [74]:
bins = range(47)
hist, edges = np.histogram(n_draws, bins=bins, density=True)


In [78]:
histogram = make_plot('Expected number of draws for a Jack', hist, edges)

In [79]:
show(histogram)

In [21]:
ps, not_ps, pcs = [], [], []
c = 52
k = 4
def calc_p(m):
    p_jack = k / (c - m)
    if c - k - m >= 0:
        p_not_jack = (math.factorial(c-k) / math.factorial(c)) * (math.factorial(c-m)/math.factorial(c-k-m)) 
    else:
        p_not_jack = 0
    return p_jack, p_not_jack, p_jack * p_not_jack

In [24]:

p = figure(width=400, height=400)

p.circle(list(range(c-1)), ps, size=10, color="dodgerblue", alpha=0.5)
p.circle(list(range(c-1)), not_ps, size=10, color='firebrick', alpha=0.5)
p.circle(list(range(c-1)), pcs, size=10, color='green', alpha=0.5)
show(p)

In [6]:
import rioxarray as rxr

In [7]:
dem = '/media/danbot/Samsung_T5/geospatial_data/DEM_data/EarthEnv_DEM90/'
dem_folders = os.listdir(dem)


In [13]:
raster = rxr.open_rasterio(dem+dem_folders[0]+'/'+dem_folders[0]+'.bil')

In [19]:
raster.rio.crs.to_wkt()

'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433],AXIS["Longitude",EAST],AXIS["Latitude",NORTH]]'

In [27]:
new = new.replace(tzinfo=pytz.utc)
new.strftime('%Y-%m-%d %H:%M')

'2022-03-07 09:51'

In [4]:
data_dir = '/media/danbot/Samsung_T5/geospatial_data/'
hysets_dir = os.path.join(data_dir, 'HYSETS_data/')
hydat_dir = os.path.join(data_dir, 'hydat_db/')
dem_dir = os.path.join(data_dir, 'DEM_data/')
processed_dem_dir = os.path.join(dem_dir, 'processed_dem/')
# dem_fpath_1km = os.path.join(dem_dir, 'watershed_group_tiles_merged_1km/')
# dem_fpath_10km = os.path.join(dem_dir, 'watershed_group_tiles_merged_10km/')

# root_path = '/home/danbot/Documents/code/thesis_code/validate_hysets/'
# updated basins from December 2021 (dave Hutchinson) have path formats like: 
# WSC_data/2021-12-basins/all/07AA001/basin/07AA001_DrainageBasin_BassinDeDrainage.shp
wsc_basin_dir = data_dir + 'WSC_data/2021-12-basins/all/'

# other data sources
glhymps_fpath = data_dir + 'GLHYMPS/GLHYMPS.gdb'

# snow and land use / land cover
nalcms_fpath = data_dir + 'NALCMS_NA_2010/NA_NALCMS_2010_v2_land_cover_30m/' + 'NA_NALCMS_2010_v2_land_cover_30m.tif'

# where to save results of validation
processed_data_output_path = '/media/danbot/T7 Touch/thesis_data/processed_stations/'
processed_polygons_out_path = '/home/danbot/Documents/code/hysets_validation/processed_data/derived_basins/'

In [27]:
# file containing derived watershed properties used in hysets
hysets_props_fname = 'HYSETS_watershed_properties.txt'
# import the set of derived watershed properties from hysets
hysets_props = pd.read_csv(hysets_dir + hysets_props_fname, delimiter=';')


In [37]:
shapes = sorted(os.listdir(processed_polygons_out_path + 'pysheds'))

for s in shapes:
    stn = s.split('_')[0]
    resolution = s.split('.')[0].split('_')[-1]
    
    hs_info = hysets_props[hysets_props['Official_ID'] == stn]
    hs_area = hs_info['Drainage_Area_km2'].values[0]
    hs_area_flag = hs_info['Flag_GSIM_boundaries'] == 1
    hs_bounds = hs_info['Flag_Artificial_Boundaries'] == 1
    hs_perim = hs_info['Perimeter'].values[0]
    
    
    derived_basin = gpd.read_file(processed_polygons_out_path + f'pysheds/{s}', driver='GeoJSON')
    area = derived_basin.geometry.area.values[0] / 1E6
    perim = derived_basin.geometry.length.values[0] / 1E3
    p_type = derived_basin.geometry.type.values[0]
    n_objs = len(derived_basin.geometry.explode(index_parts=False))
    
    if n_objs > 1:
        print(stn)
        area = derived_basin.geometry.area.values[0] / 1E6
        perim = derived_basin.geometry.length.values[0] / 1E3
        p_type = derived_basin.geometry.type.values[0]
        n_objs = len(derived_basin.geometry.explode(index_parts=False))
        
        area1 = sum(derived_basin.geometry.area.values) / 1E6
        perim1 = sum(derived_basin.geometry.length.values) / 1E3
        
        n_objs1 = len(derived_basin.geometry.explode(index_parts=False))
        
        print(f'    Hysets: {hs_area:.1f} km^2 {hs_perim:.1f} km')
        print(f'    Derived ({p_type}): {area:.1f} km^2 {perim:.1f} km {n_objs} object(s) {resolution} res')
        print(f'    Derived ({p_type}): {area1:.1f} km^2 {perim1:.1f} km ')
    # print(asdfd)
    
    
    

07EE010
    Hysets: 3711.8 km^2 483.3 km
    Derived (MultiPolygon): 1721.4 km^2 475.7 km 2 object(s) low res
    Derived (MultiPolygon): 1721.4 km^2 475.7 km 
07FD001
    Hysets: 3634.8 km^2 393.3 km
    Derived (MultiPolygon): 3582.4 km^2 529.2 km 2 object(s) low res
    Derived (MultiPolygon): 3582.4 km^2 529.2 km 
07GB002
    Hysets: 3300.0 km^2 402.9 km
    Derived (MultiPolygon): 3259.3 km^2 512.6 km 3 object(s) low res
    Derived (MultiPolygon): 3259.3 km^2 512.6 km 
07GB003
    Hysets: 3370.9 km^2 405.5 km
    Derived (MultiPolygon): 3406.8 km^2 535.4 km 3 object(s) low res
    Derived (MultiPolygon): 3406.8 km^2 535.4 km 
07GD002
    Hysets: 666.0 km^2 139.3 km
    Derived (MultiPolygon): 276.4 km^2 196.9 km 2 object(s) hi res
    Derived (MultiPolygon): 276.4 km^2 196.9 km 
07GD003
    Hysets: 1610.0 km^2 272.0 km
    Derived (MultiPolygon): 598.3 km^2 231.8 km 2 object(s) hi res
    Derived (MultiPolygon): 598.3 km^2 231.8 km 
07GD003
    Hysets: 1610.0 km^2 272.0 km
    De